In [1]:
import geopandas as gpd
import fiona
import sys
sys.path.append('/opt/conda/share/proj')

file_path = '/mnt/diskstation/data/soil_points/Netherlands/brobhrpvolledigeset.gpkg'
layers = fiona.listlayers(file_path)

# gdf = gpd.read_file(file_path)


ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed


In [3]:
# NL SOM
investigated_interval = gpd.read_file(file_path, layer = 'investigated_interval')
borehole_sample_analysis = gpd.read_file(file_path, layer = 'borehole_sample_analysis')
borehole_research = gpd.read_file(file_path, layer = 'borehole_research')
om = gpd.read_file(file_path, layer = 'organic_matter_content_determination')
om1 = om.merge(investigated_interval, left_on='investigated_interval_fk', right_index=True)
om2 = om1.merge(borehole_sample_analysis, left_on='borehole_sample_analysis_fk', right_index=True)
om2 = om2.drop(columns='geometry')
om3 = om2.merge(borehole_research, left_on='borehole_research_fk', right_index=True)
om3 = om3.drop(columns=['investigated_interval_fk','geometry_x','borehole_sample_analysis_fk','geometry_y', 'borehole_research_fk',
                       'site_characteristic_determined','litter_layer_investigated', 'borehole_geophysically_logged',
                        'borehole_samples_described', 'borehole_samples_analyzed','borehole_samples_photographed','characteristic_modelled',
                        'investigated_area','lutum_correction_applied','free_iron_correction_applied','location_specific_y','location_specific_x',
                        'horizon_code','analysis_type','quality_regime','delivery_accountable_party', 'delivery_context', 'survey_purpose',
                        'discipline','reporting_limit','applied_transformation'])
om3 = om3.loc[om3['determination_method']!='onbekend']

om3 = gpd.GeoDataFrame(om3)
om3.to_crs(epsg='4326')
om3.to_file('/mnt/primus/xuemeng_tmp_harbour/soc_eu/netherlands/som_nl.4326.gpkg',driver='GPKG')

In [3]:
# NL dry bulk density
bulk_density = gpd.read_file(file_path, layer='dry_bulk_density_determination')
bulk_density = bulk_density.merge(investigated_interval, left_on='investigated_interval_fk', right_index=True)
bulk_density = bulk_density.merge(borehole_sample_analysis, left_on='borehole_sample_analysis_fk', right_index=True)
bulk_density = bulk_density.drop(columns='geometry')
bulk_density = bulk_density.merge(borehole_research, left_on='borehole_research_fk', right_index=True)

bulk_density = bulk_density.drop(columns=['investigated_interval_fk', 'ring_diameter', 'ring_height',
                                'material_irregularity', 'geometry_x', 'borehole_sample_analysis_fk',
                                'location_specific_x','characteristic_modelled', 'horizon_code', 'geometry_y',
                                'borehole_research_fk', 'analysis_type','location_specific_y', 'investigated_area', 'bro_id', 'quality_regime',
                                'delivery_accountable_party', 'delivery_context', 'survey_purpose',
                                'discipline', 'site_characteristic_determined','applied_transformation',
                                'litter_layer_investigated', 'borehole_geophysically_logged',
                                'borehole_samples_described', 'borehole_samples_analyzed','borehole_samples_photographed'])

bulk_density = gpd.GeoDataFrame(bulk_density)
bulk_density.to_crs(epsg='4326')
bulk_density.to_file('/mnt/primus/xuemeng_tmp_harbour/soc_eu/netherlands/bulk.density_nl.4326.gpkg',driver='GPKG')

In [58]:
# NL, texture
tsmall = gpd.read_file(file_path, layer='standard_distribution_fraction_smaller50um')
tlarge = gpd.read_file(file_path, layer='standard_distribution_fraction63to2000um')
tbasic = gpd.read_file(file_path, layer='basic_particle_size_distribution')
texture = tsmall.merge(tbasic,left_on='basic_particle_size_distribution_fk',right_index=True)
texture = texture.merge(tlarge,on='basic_particle_size_distribution_fk')
texture = texture.drop(columns=['geometry','geometry_y'])
size = gpd.read_file(file_path, layer='particle_size_distribution_determination')
texture = texture.merge(size,left_on='particle_size_distribution_determination_fk',right_index=True)
texture = texture.drop(columns=['geometry','geometry_x'])
texture = texture.merge(investigated_interval, left_on='investigated_interval_fk', right_index=True)
texture = texture.drop(columns='geometry')
texture = texture.merge(borehole_sample_analysis, left_on='borehole_sample_analysis_fk', right_index=True)
texture = texture.drop(columns='geometry')
texture = texture.merge(borehole_research, left_on='borehole_research_fk', right_index=True)

texture['clay'] = texture['fraction0to2um']
texture['silt'] = texture['fraction_smaller50um'] - texture['fraction0to2um']
texture['sand'] = texture['fraction50to63um'] + texture['fraction63to2000um']

texture = texture.drop(columns=['basic_particle_size_distribution_fk', 
       'particle_size_distribution_determination_fk', 
       'investigated_interval_fk', 'particle_size_distribution_standardised',
       'fraction_distribution', 
       'performance_irregularity', 'borehole_sample_analysis_fk',
        'location_specific_x',
       'characteristic_modelled', 'horizon_code', 'borehole_research_fk',
       'analysis_type', 'location_specific_y',
       'investigated_area', 'bro_id', 'quality_regime',
       'delivery_accountable_party', 'delivery_context', 'survey_purpose',
       'discipline', 'site_characteristic_determined',
       'litter_layer_investigated', 'borehole_geophysically_logged',
       'borehole_samples_described', 'borehole_samples_analyzed',
       'borehole_samples_photographed', 'applied_transformation','fraction0to2um', 'fraction2to16um', 'fraction16to50um',
       'fraction_smaller50um', 'fraction50to63um', 'fraction63to2000um',
       'fraction63to105um', 'fraction105to210um', 'fraction210to420um',
       'fraction420to2000um'])

# texture['check'] = texture['clay'] + texture['silt'] + texture['sand'] 
# texture['check']

texture = gpd.GeoDataFrame(texture)
texture.to_crs(epsg='4326')
texture.to_file('/mnt/primus/xuemeng_tmp_harbour/soc_eu/netherlands/texture_nl.4326.gpkg',driver='GPKG')


In [68]:
texture = gpd.read_file('/mnt/primus/xuemeng_tmp_harbour/soc_eu/netherlands/texture_nl.4326.gpkg')
texture['texture_method'] = texture['determination_procedure'] + '-' + texture['determination_method'] + '-' +  texture['dispersion_method']
texture = texture.drop(columns = ['determination_procedure', 'determination_method','dispersion_method','analysis_report_date'])
texture = texture.dropna(subset='texture_method')
texture = texture.loc[texture['texture_method']=='NEN5753v2018plusWENR-natDroogZevenPipet-roerenDispersiemiddel']

bd = gpd.read_file('/mnt/primus/xuemeng_tmp_harbour/soc_eu/netherlands/bulk.density_nl.4326.gpkg')
bd['bd_method'] = bd['determination_procedure'] + '-' + bd['determination_method'] + '-' + bd['drying_temperature'] + '-' + bd['volume_water_saturated']
bd = bd.drop(columns = ['determination_procedure', 'determination_method', 'drying_temperature','volume_water_saturated','analysis_report_date'])
bd = bd.loc[bd['bd_method'] == 'ISO11272v2017plusWENR-drogenOven-105graden-ja']

som = gpd.read_file('/mnt/primus/xuemeng_tmp_harbour/soc_eu/netherlands/som_nl.4326.gpkg')
som['som_method'] = som['determination_procedure'] + '-' + som['determination_method']
som = som.drop(columns = ['determination_procedure', 'determination_method','analysis_report_date','bro_id'])

In [72]:
nl = texture.merge(bd, how='outer', on=['begin_depth', 'end_depth', 'research_report_date', 'geometry'])
nl = nl.merge(som, how='outer', on=['begin_depth', 'end_depth', 'research_report_date', 'geometry'])
nl = nl.drop(columns = ['texture_method', 'bd_method', 'som_method'])


In [75]:
nl.to_file('/mnt/primus/xuemeng_tmp_harbour/soc_eu/netherlands/nl.4326.gpkg',driver='GPKG')

In [5]:
sclass

borehole_sample_description_fk         code_group  \
0                                    1          kleigrond   
1                                    2          kleigrond   
2                                    3       moerigeGrond   
3                                    4        podzolgrond   
4                                    5          kleigrond   
...                                ...                ...   
342108                          342109  zandgrondKalkloos   
342109                          342110  zandgrondKalkloos   
342110                          342111  zandgrondKalkloos   
342111                          342112  zandgrondKalkloos   
342112                          342113  zandgrondKalkloos   

             classification_code feature_top                 soil_class  \
0         R5x  315  b                   geen     ooivaaggrondRivierklei   
1         R5p  235  bg12                geen  poldervaaggrondRivierklei   
2         2m   431                      geen             dampodzolgrond   
3         2v   433   x10                geen     haarpodzolgrondZanddek   
4         M4o  233  c                   geen       woudeerdgrondZeeklei   
...                          ...         ...                        ...   
342108    4k   433                      geen        zwarteBeekeerdgrond   
342109    5k   433   g11                geen              vlakvaaggrond   
342110    5t   432                      geen              duinvaaggrond   
342111    5k   432                      geen              vlakvaaggrond   
342112    5k   432                      geen              vlakvaaggrond   

                  texture_class peat_class subsoil_peat  lower_boundary_peat  \
0                     kleiLicht       None         None                  NaN   
1                    zavelZwaar       None         None                  NaN   
2          zandLeemarmMatigFijn       None         None                  NaN   
3       zandSterkLemigMatigFijn       None         None                  NaN   
4                    zavelZwaar       None         None                  NaN   
...                         ...        ...          ...                  ...   
342108  zandSterkLemigMatigFijn       None         None                  NaN   
342109  zandSterkLemigMatigFijn       None         None                  NaN   
342110   zandZwakLemigMatigFijn       None         None                  NaN   
342111   zandZwakLemigMatigFijn       None         None                  NaN   
342112   zandZwakLemigMatigFijn       None         None                  NaN   

       subsoil_duin_vague_soil   texture_profile carbonate_profile  \
0                         None          homogeen           kalkarm   
1                         None          homogeen           kalkarm   
2                         None              None              None   
3                         None              None              None   
4                         None  opKleiTussenlaag          kalkloos   
...                        ...               ...               ...   
342108                    None              None              None   
342109                    None              None              None   
342110             nietBepaald              None              None   
342111                    None              None              None   
342112                    None              None              None   

       reworking_class groundwater_table_class anomalous_groundwater_regime  \
0                 geen                    VIIo                         None   
1                 geen                    VIIo                         None   
2                 geen                     VIo                         None   
3                 geen                   VIIId                         None   
4                 geen                    IIIa                         None   
...                ...                     ...                          ...   
342108            g

In [ ]:
# sclass = gpd.read_file(file_path,layer='soil_classification')
# borehole_sample_description = gpd.read_file(file_path, layer = 'borehole_sample_description')
# sclass = sclass.merge(borehole_sample_description, left_on='borehole_sample_description_fk', right_index=True)
# borehole_research = gpd.read_file(file_path, layer = 'borehole_research')
# sclass = sclass.merge(borehole_research, left_on='borehole_research_fk', right_index=True)
# sclass = sclass.drop(columns=['borehole_sample_description_fk', 'geometry_x', 'borehole_research_fk',
#        'description_completion_date', 'description_method',
#        'description_location', 'fraction_distribution_determined',
#        'lower_boundary_sand_fraction', 'geometry_y', 'bro_id',
#        'quality_regime', 'delivery_accountable_party', 'delivery_context',
#        'survey_purpose', 'discipline', 
#        'site_characteristic_determined', 'litter_layer_investigated',
#        'borehole_geophysically_logged', 'borehole_samples_described',
#        'borehole_samples_analyzed', 'borehole_samples_photographed',
#        'applied_transformation'])

sclass = gpd.GeoDataFrame(sclass)
sclass.to_crs(epsg='4326')
sclass.to_file('/mnt/primus/xuemeng_tmp_harbour/soc_eu/netherlands/soil_classification_nl.gpkg',driver='GPKG')

In [170]:
# # layer soil type
# soil_type = gpd.read_file(file_path, layer='soil_type')
# layer_component = gpd.read_file(file_path, layer = 'layer_component')
# soil_type = soil_type.merge(layer_component,left_on='layer_component_fk',right_index=True)
# soil_layer = gpd.read_file(file_path, layer = 'soil_layer')
# soil_type = soil_type.merge(soil_layer,left_on='soil_layer_fk',right_index=True)
# soil_type = soil_type.drop(columns=['geometry_x','geometry_y'])
# descriptive_borehole_log = gpd.read_file(file_path, layer = 'descriptive_borehole_log')
# soil_type = soil_type.merge(descriptive_borehole_log, left_on='descriptive_borehole_log_fk',right_index=True)
# borehole_sample_description = gpd.read_file(file_path, layer = 'borehole_sample_description')
# soil_type = soil_type.merge(borehole_sample_description, left_on='borehole_sample_description_fk',right_index=True)
# soil_type = soil_type.drop(columns='geometry')
# soil_type = soil_type.merge(borehole_research, left_on='borehole_research_fk', right_index=True)

# soil_type = soil_type.drop(columns=['layer_component_fk', 'soil_layer_fk', 'volume_percentage','horizon_code', 'saturated_permeability',
#                                     'descriptive_borehole_log_fk', 'upper_boundary', 'lower_boundary','number_of_layer_components', 'geometry_x',
#                                     'borehole_sample_description_fk', 'sample_quality','mean_highest_groundwater_level_reached',
#                                     'root_penetrable_depth_reached', 'root_penetrable_depth',
#                                     'mean_highest_groundwater_level', 'mean_lowest_groundwater_level','geometry_y', 'borehole_research_fk', 'description_completion_date',
#                                     'description_method', 'description_location','fraction_distribution_determined', 'lower_boundary_sand_fraction',
#                                     'bro_id', 'quality_regime', 'delivery_accountable_party','delivery_context', 'survey_purpose', 'discipline',
#                                     'research_report_date', 'site_characteristic_determined','litter_layer_investigated', 'borehole_geophysically_logged',
#                                     'borehole_samples_described', 'borehole_samples_analyzed','borehole_samples_photographed', 'applied_transformation'])

# soil_type = gpd.GeoDataFrame(soil_type)
# soil_type.to_crs(epsg='4326')
# soil_type.to_file('soil.type.layer_nl.4326.gpkg',driver='GPKG')